In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import os

In [8]:
def leer_matriz(nombre_archivo):
    matriz = []
    with open(nombre_archivo, 'r') as archivo:
        archivo.readline()
        archivo.readline()
        for linea in archivo:
            fila = [float(valor) for valor in linea.strip().split()]
            matriz.append(fila)
    return matriz

In [2]:
dir = '/home/user/Desktop/Datos/'
#dir = '/Users/antoniomendez/Desktop/Tesis/Datos/datos_limpios/'

In [3]:
# Load data
data_2019 = pd.read_csv(f'{dir}mibici/2019.csv')
data = data_2019[data_2019['Inicio_del_viaje'].str.startswith('2019-01')]
del data_2019

In [4]:
data.head(3)

,Viaje_Id,Usuario_Id,Genero,Año_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id,Tiempo_viaje
0,8901295,341162,M,1992,2019-01-01 00:00:00,2019-01-01 00:16:09,50,54,16.150000
1,8901296,413443,M,1985,2019-01-01 06:01:05,2019-01-01 06:16:27,38,194,15.366667
2,8901297,444232,M,1995,2019-01-01 06:01:19,2019-01-01 06:09:55,86,18,8.600000


In [9]:
estaciones = leer_matriz(f'{dir}Adj_mibici/matrices_estaciones/est_2019.txt')

In [26]:
estaciones

[[2.0, 20.666378, -103.34882],
 [3.0, 20.667228, -103.366],
 [4.0, 20.66769, -103.368252],
 [5.0, 20.6918466, -103.3625487],
 [6.0, 20.6811575, -103.339363],
 [8.0, 20.680717, -103.344415],
 [9.0, 20.666771, -103.350562],
 [10.0, 20.681871, -103.350396],
 [11.0, 20.68207685, -103.353835],
 [12.0, 20.6818519, -103.3573381],
 [13.0, 20.681081, -103.360099],
 [14.0, 20.6820683, -103.3629218],
 [15.0, 20.667623, -103.370499],
 [16.0, 20.6821447, -103.3659297],
 [17.0, 20.683191, -103.369349],
 [18.0, 20.6824637, -103.3726232],
 [19.0, 20.679086, -103.372841],
 [20.0, 20.679296, -103.370658],
 [21.0, 20.67881367, -103.3680584],
 [23.0, 20.67966, -103.362224],
 [24.0, 20.679389, -103.35913],
 [25.0, 20.679309, -103.356281],
 [26.0, 20.678777, -103.354035],
 [27.0, 20.680227, -103.350146],
 [28.0, 20.680478, -103.348377],
 [29.0, 20.679861, -103.345198],
 [30.0, 20.6840145, -103.3612673],
 [31.0, 20.679899, -103.339438],
 [32.0, 20.675461, -103.367501],
 [33.0, 20.685652, -103.368215],
 [34.0

In [27]:
def generar_colores(n):
    # Obtener un mapa de colores 'hsv' con n colores distintos
    colores = plt.cm.get_cmap('coolwarm', n)
    # Generar el vector de colores
    return [colores(i) for i in range(n)]

def encontrar_estacion(est, matriz):
    for i in range(len(matriz)):
        if matriz[i][0] == est:
            return matriz[i][1], matriz[i][2]
    return None, None

def user_flow(user_data, estaciones, directory, zoom = True):
    viajes_por_estaciones = user_data.groupby([user_data[['Origen_Id', 'Destino_Id']].min(axis=1), user_data[['Origen_Id', 'Destino_Id']].max(axis=1)]).size().reset_index(name='counts')
    viajes_por_estaciones.columns = ['Estacion_A', 'Estacion_B', 'counts']
    latitudes = []
    longitudes = []
    est_plotted = set()
    plt.figure(figsize=(10, 10))

    for i, viaje, in viajes_por_estaciones.iterrows():
        est_A, est_B = viaje['Estacion_A'], viaje['Estacion_B']
        lat_A, lon_A = encontrar_estacion(est_A, estaciones)
        lat_B, lon_B = encontrar_estacion(est_B, estaciones)
        latitudes.extend([lat_A, lat_B])
        longitudes.extend([lon_A, lon_B])

        transparency = 0.5 * viaje['counts'] / viajes_por_estaciones['counts'].max()

        if est_A == est_B:
            plt.scatter(lon_A, lat_A, marker='*', s=100, color = 'b')
            est_plotted.add(est_A)
        else:
            plt.plot([lon_A, lon_B], [lat_A, lat_B], color='b', alpha = transparency)

    for i in range(len(estaciones)):
        estacion = estaciones[i]
        est = estacion[0]
        lat, lon = estacion[1], estacion[2]
        if est not in est_plotted:
            plt.scatter(lon, lat, marker='o', s=10, color='black')

    # Ajusta los límites del gráfico para centrar en la media
    if len(latitudes) > 0:
        if zoom:
            lon_max = np.max(longitudes)
            lon_min = np.min(longitudes)
            lat_max = np.max(latitudes)
            lat_min = np.min(latitudes)
            aux_lon = np.abs(lon_max - lon_min) / 10
            aux_lat = np.abs(lat_max - lat_min) / 10
            if aux_lon == 0:
                aux_lon = 0.01
            if aux_lat == 0:
                aux_lat = 0.005
            plt.xlim(lon_min - aux_lon, lon_max + aux_lon)
            plt.ylim(lat_min - aux_lat, lat_max + aux_lat)

        user = user_data['Usuario_Id'].iloc[0]
        genre = user_data['Genero'].iloc[0]
        age = 2024 - user_data['Año_de_nacimiento'].iloc[0]
        date = user_data['Inicio_del_viaje'].iloc[0]
        date = date.split(' ')[0]
        total_viajes = viajes_por_estaciones['counts'].sum()

        plt.title(f'Usuario: {user}, Género: {genre}, Edad: {age}, Fecha: {date}, Total de viajes: {total_viajes}')
        # Agrupar elementos en el legend si hay más de 42
        plt.grid()
        """handles, labels = plt.gca().get_legend_handles_labels()
        if len(handles) > 42:
            grouped_handles = handles[:41]
            grouped_labels = labels[:41]
            grouped_handles.append(plt.Line2D([0], [0], color='black', lw=2))
            grouped_labels.append(f'Y {len(handles) - 41} más...')
            plt.legend(grouped_handles, grouped_labels, loc='best')
        else:
            plt.legend(loc='best')"""
        #plt.legend(loc='best')
        directory = directory if zoom else f'{directory}_nozoom'
        if not os.path.exists(directory):
            os.makedirs(directory)
        if zoom:
            plt.savefig(f'{directory}/usuario_{user}_trayectorias.png')
        else:
            plt.savefig(f'{directory}_nozoom/usuario_{user}_trayectorias.png')
        plt.close()
        plt.clf()

In [ ]:
dates = [f'2019-01-{i:02d}' for i in range(1, 2)]

for date in dates:
    current_data = data[data['Inicio_del_viaje'].str.startswith(date)]
    current_users = current_data['Usuario_Id'].unique()
    print(f'Fecha: {date}, cantidad de usuarios en el día: {len(current_users)}')
    for user in current_users:
        user_data = current_data[current_data['Usuario_Id'] == user]
        user_flow(user_data, estaciones, f'/home/user/Desktop/Graficas/{date}', zoom = True)
        

Fecha: 2019-01-01, cantidad de usuarios en el día: 1268
